# CNN for seismic facies classification

## Imports and dataset loading

In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import h5py
import keras
import k3d
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from livelossplot.keras import PlotLossesCallback
from sklearn.metrics import confusion_matrix, f1_score
from sklearn.utils import class_weight
from ipywidgets import interact, widgets

import src

## Dataset visualization

In [ ]:
with h5py.File('../data/interim/stanford6_truncated_rgb.h5', 'r') as dataset:
    x_train_original = np.array(dataset['train/X'])
    y_train_original = np.array(dataset['train/Y'])
    x_test_original = np.array(dataset['test/X'])
    y_test_original = np.array(dataset['test/Y'])

classnames = {
    0: 'Floodplain',
    1: 'Pointbar',
    2: 'Channel',
    3: 'Boundary',
}    
    
m = x_train_original.shape[0]
num_classes = 4

frequencies = src.class_frequency(y_train_original, num_classes)
print(f'Training class frequencies: {frequencies}')
print(f'Test class frequencies: {src.class_frequency(y_test_original, num_classes)}')

nrows, ncols = 2, 5
idx = np.random.choice(m, nrows * ncols)
fig = plt.figure(figsize=(12, 5))
for i in range(1, nrows * ncols +1):
    fig.add_subplot(nrows, ncols, i)
    plt.imshow(x_train_original[idx[i-1]])
    plt.title(f'class: {classnames[y_train_original[idx[i-1]][0]]}')
    plt.xticks([])
    plt.yticks([])   

## Dataset normalization

In [ ]:
from imblearn.over_sampling import ADASYN

x_train_resampled, y_train_resampled = ADASYN().fit_resample(np.reshape(x_train_original, (m, np.product(x_train_original.shape[1:]))), y_train_original)
x_train_resampled = np.reshape(x_train_resampled, (x_train_resampled.shape[0], *x_train_original.shape[1:]))
print(src.class_frequency(y_train_resampled, num_classes))

x_train = x_train_resampled / 255
y_train = keras.utils.to_categorical(y_train_resampled, num_classes)
x_test = x_test_original / 255
y_test = keras.utils.to_categorical(y_test_original, num_classes)



In [ ]:
x_train = x_train_original / 255
y_train = keras.utils.to_categorical(y_train_original, num_classes)
x_test = x_test_original / 255
y_test = keras.utils.to_categorical(y_test_original, num_classes)

## The model

In [ ]:
input_shape = x_train.shape[1:]

model = Sequential()

model.add(Conv2D(32, (3, 3), padding='same', input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Flatten())

model.add(Dense(256))
model.add(Activation('relu'))

model.add(Dense(num_classes))
model.add(Activation('softmax'))

opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

model.compile(
    loss='categorical_crossentropy',
    optimizer=opt,
    metrics=['accuracy']
)

model.summary()

## Model training

In [ ]:
batch_size = 32
epochs = 180

history = model.fit(
    x_train, y_train,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
    shuffle=True,
    callbacks=[PlotLossesCallback()],
)

## Model validation

In [ ]:
scores = model.evaluate(x_test, y_test, verbose=1)
for i, score in enumerate(scores):
    print(f'Test {model.metrics_names[i]}: {score}')
    
predict_class = np.argmax(model.predict(x_test), axis=1)
print(f'F1-score: {f1_score(y_test_original, predict_class, average="weighted")}')
matrix = confusion_matrix(y_test_original, predict_class)
matrix = matrix.astype('float') / matrix.sum(axis=1)[:, np.newaxis]
      
src.plot_confusion_matrix(matrix, classnames.values(), title="Confusion matrix")

In [ ]:
def plot_section(z=0):
    plt.subplot(121)
    plt.imshow(y_test_original.reshape(119, 169, 88)[:,:,z].T)
    plt.subplot(122)
    plt.imshow(predict_class.reshape(119, 169, 88)[:,:,z].T)
    
interact(plot_section, z=widgets.IntSlider(min=0,max=87,step=1,value=0))

In [ ]:
color_map = (0x3A528B, 0x20908C, 0xFDE724)

plot = k3d.plot()
obj = k3d.voxels(predict_class.reshape(119, 169, 88).T, color_map, compression_level=1)
plot += obj
plot.display()